# Point cloud generation from mesh

## Dedenencies

* numpy
* matplotlib
* numpy-stl
  - pip install numpy-stl
  
## Decription
Convert mesh to point cloud

* 3D CAD Model
  - ![CAD](./image/cad_model.png)

* Mesh Model
  - ![mesh](./image/mesh_model.png)

* Point cloud
  - ![pointcloud](./image/pointcloud.png)

  

## Load mesh files


In [4]:
import numpy as np
from stl import mesh

print mesh.__file__
mymesh = mesh.Mesh.from_file('/media/kyungpyo/SmartCarContest/MasterThesisProject/new_dataset/dataset/stl_files/00_total/car/3d-Car-Model.stl')

/home/kyungpyo/.local/lib/python2.7/site-packages/stl/mesh.pyc


## Sampling point cloud from mesh (test)

In [5]:
"""
Reference: https://medium.com/@daviddelaiglesiacastro/3d-point-cloud-generation-from-3d-triangular-mesh-bbb602ecf238
"""
n = 10000

def triangle_area_multi(v1, v2, v3):
    return 0.5 * np.linalg.norm( np.cross(v2-v1, v3-v1), axis = 1)

areas = triangle_area_multi(mymesh.v0, mymesh.v1, mymesh.v2)

probabilities = areas / areas.sum()

## select triangles to sample point cloud
weighted_random_indices = np.random.choice( range(len(areas)), size=n, p=probabilities)

v0_xyz = mymesh.v0[weighted_random_indices]
v1_xyz = mymesh.v1[weighted_random_indices]
v2_xyz = mymesh.v2[weighted_random_indices]

u = np.random.rand(n,1)
v = np.random.rand(n,1)
is_a_problem = u + v > 1

u[is_a_problem] = 1 - u[is_a_problem]
v[is_a_problem] = 1 - v[is_a_problem]

w = 1 - (u + v)

result_xyz = (v0_xyz * u) + (v1_xyz * v) + (v2_xyz * w)


ValueError: a must be non-empty

In [7]:
mymesh.v0

array([], shape=(0, 3), dtype=float32)

## Visualization

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

def display_point_cloud_box_ax_test(ax, pts):
    
    # display point cloud
    ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=5, alpha = 1.0)
    
    # display center point
    cp = np.mean(pts, axis = 0)
    ax.scatter(cp[0], cp[1], cp[2], c='r', s=5)
    
    # display bounding box   
    cube_definition_array = [
        np.array(list(item))
        for item in find_cube_definition(pts)
    ]
    points = []
    points += cube_definition_array
    vectors = [
        cube_definition_array[1] - cube_definition_array[0],
        cube_definition_array[2] - cube_definition_array[0],
        cube_definition_array[3] - cube_definition_array[0]
    ]

    points += [cube_definition_array[0] + vectors[0] + vectors[1]]
    points += [cube_definition_array[0] + vectors[0] + vectors[2]]
    points += [cube_definition_array[0] + vectors[1] + vectors[2]]
    points += [cube_definition_array[0] + vectors[0] + vectors[1] + vectors[2]]

    points = np.array(points)

    edges = [
        [points[0], points[3], points[5], points[1]],
        [points[1], points[5], points[7], points[4]],
        [points[4], points[2], points[6], points[7]],
        [points[2], points[6], points[3], points[0]],
        [points[0], points[2], points[4], points[1]],
        [points[3], points[6], points[7], points[5]]
    ]

    faces = Poly3DCollection(edges, linewidths=0.1, edgecolors='k')
    faces.set_facecolor((0,0,1,0.05))
    
    ax.add_collection3d(faces)
    
    # Edges of the cube
    ax.scatter(points[:,0], points[:,1], points[:,2], s=0)

    # make axis equal
    p_min = np.min(points, axis=0)
    p_max = np.max(points, axis=0)
    max_range = np.array([p_max[0]-p_min[0], p_max[1]-p_min[1], p_max[2]-p_min[2]]).max() / 2.0

    mid_x = (p_max[0]+p_min[0]) * 0.5
    mid_y = (p_max[1]+p_min[1]) * 0.5
    mid_z = (p_max[2]+p_min[2]) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    return ax


def find_cube_definition(pts):
    p_min = np.min(pts, axis = 0)
    p_max = np.max(pts, axis = 0)
    return [ (p_min[0],p_min[1],p_min[2]), 
            (p_max[0],p_min[1],p_min[2]), 
            (p_min[0],p_max[1],p_min[2]), 
            (p_min[0],p_min[1],p_max[2])]
def resample_point_cloud(pts, resample_ratio):
    step = int(1./resample_ratio)
    idx = range(0, len(pts), step)
    return pts[idx]

In [21]:
%matplotlib qt
%matplotlib qt
%matplotlib qt
%matplotlib qt
fig = plt.figure(figsize = (10,10))

ax = fig.add_subplot(1,1,1, projection='3d')

display_point_cloud_box_ax_test(ax, result_xyz)
ax.view_init(elev=100, azim=250)

plt.axis('off')
plt.show()

## Function implementation

In [2]:
def pointcloud_from_mesh(v0,v1,v2, n):
    """
    input:
        v0, v1, v2: vertex list of meshs
        n = number of generated points
    output:
        point cloud: n points
    reference:
        https://medium.com/@daviddelaiglesiacastro/3d-point-cloud-generation-from-3d-triangular-mesh-bbb602ecf238
    """
    
    ## Sample point cloud uniformly using areas
    areas = 0.5 * np.linalg.norm( np.cross(v1-v0, v2-v0), axis = 1 )
    probabilities = areas / areas.sum()

    # select triangles to sample point cloud
    weighted_random_indices = np.random.choice( range(len(areas)), size=n, p=probabilities)

    v0_xyz = v0[weighted_random_indices]
    v1_xyz = v1[weighted_random_indices]
    v2_xyz = v2[weighted_random_indices]

    ## random sample
    u = np.random.rand(n,1)
    v = np.random.rand(n,1)
    is_a_problem = u + v > 1

    u[is_a_problem] = 1 - u[is_a_problem]
    v[is_a_problem] = 1 - v[is_a_problem]

    w = 1 - (u + v)

    return (v0_xyz * u) + (v1_xyz * v) + (v2_xyz * w)

## Run all files

In [11]:
import os
import numpy as np
from stl import mesh

class_name = 'car'

root_path = os.path.abspath('./dataset/stl_files/00_total/car/')
out_path = os.path.abspath('./dataset/point_dataset/car')
try:
    if not(os.path.isdir(out_path)):
        os.makedirs(os.path.join(out_path))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
    
"""
Not converted files
"""
root_nc_path = os.path.join(root_path, 'not_conversion')
try:
    if not(os.path.isdir(root_nc_path)):
        os.makedirs(os.path.join(root_nc_path))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
    

## find ".stl" files
fname = []
for root,d_names,f_names in os.walk(root_path):
    for f in f_names:
        if f.split('.')[-1] == 'stl':
            fname.append(os.path.join(root, f))

## convert to numpy point cloud
for i in range ( len ( fname ) ):
    print "File path:", fname[i].split("/")[-1]

    try:
    
        meshdata = mesh.Mesh.from_file(fname[i])
        
        pc = pointcloud_from_mesh(meshdata.v0, meshdata.v1, meshdata.v2, n = 100000)
    
        out_file_name = "{}_{:04}".format(class_name, i)

        out_file_path = os.path.join(out_path, out_file_name)

        np.save(out_file_path, pc)

        del pc

        print "Save file:", out_file_name 
    
    except AssertionError:
        
        meshdata_nc_path = os.path.join(root_nc_path, fname[i].split("/")[-1])
        
        os.system( "mv {} {}/".format(fname[i], root_nc_path) )
        
        continue
        
    except ValueError:
        meshdata_nc_path = os.path.join(root_nc_path, fname[i].split("/")[-1])
        
        os.system( "mv {} {}/".format(fname[i], root_nc_path) )
        
        continue


File path: 1.stl
Save file: car_0000
File path: 3d-Car-Model.stl
File path: as-1-mr2.stl
Save file: car_0002
File path: Astin-DB9.stl
Save file: car_0003
File path: Audi-TT.stl
Save file: car_0004
File path: Azerant.stl
Save file: car_0005
File path: beetle.stl
File path: BRZ4.stl
File path: bus.stl
Save file: car_0008
File path: BUS_2.stl
File path: BUS_3.stl
File path: car.stl
Save file: car_0011
File path: car_0001.stl
File path: car_0002.stl
Save file: car_0013
File path: car_0003.stl
File path: car_0004.stl
File path: car_0005.stl
File path: car_0006.stl
File path: car_0008.stl
Save file: car_0018
File path: car_0009.stl
File path: car_0010.stl
File path: car_0011.stl
File path: car_0012.stl
File path: car_0013.stl
File path: car_0014.stl
File path: car_0015.stl
File path: car_0016.stl
File path: car_0017.stl
File path: car_0018.stl
Save file: car_0028
File path: car_0019.stl
File path: car_0020.stl
File path: car_0022.stl
File path: car_0023.stl
File path: car_0024.stl
File path:

In [12]:
import os
import numpy as np
from stl import mesh

class_name = 'pedestrian'

root_path = os.path.abspath('./dataset/stl_files/00_total/pedestrian/')
out_path = os.path.abspath('./dataset/point_dataset/pedestrian')
try:
    if not(os.path.isdir(out_path)):
        os.makedirs(os.path.join(out_path))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
    
"""
Not converted files
"""
root_nc_path = os.path.join(root_path, 'not_conversion')
try:
    if not(os.path.isdir(root_nc_path)):
        os.makedirs(os.path.join(root_nc_path))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
    

## find ".stl" files
fname = []
for root,d_names,f_names in os.walk(root_path):
    for f in f_names:
        if f.split('.')[-1] == 'stl':
            fname.append(os.path.join(root, f))

## convert to numpy point cloud
for i in range ( len ( fname ) ):
    print "File path:", fname[i].split("/")[-1]

    try:
    
        meshdata = mesh.Mesh.from_file(fname[i])
        
        pc = pointcloud_from_mesh(meshdata.v0, meshdata.v1, meshdata.v2, n = 100000)
    
        out_file_name = "{}_{:04}".format(class_name, i)

        out_file_path = os.path.join(out_path, out_file_name)

        np.save(out_file_path, pc)

        del pc

        print "Save file:", out_file_name 
    
    except AssertionError:
        
        meshdata_nc_path = os.path.join(root_nc_path, fname[i].split("/")[-1])
        
        os.system( "mv {} {}/".format(fname[i], root_nc_path) )
        
        continue
        
    except ValueError:
        meshdata_nc_path = os.path.join(root_nc_path, fname[i].split("/")[-1])
        
        os.system( "mv {} {}/".format(fname[i], root_nc_path) )
        
        continue


File path: person_0047.stl
File path: person_0005.stl
File path: person_0006.stl
Save file: pedestrian_0002
File path: person_0007.stl
File path: person_0008.stl
File path: person_0009.stl
File path: person_0012.stl
File path: person_0020.stl
Save file: pedestrian_0007
File path: person_0021.stl
File path: person_0022.stl
File path: person_0024.stl
File path: person_0033.stl
File path: person_0034.stl
Save file: pedestrian_0012
File path: person_0035.stl
Save file: pedestrian_0013
File path: person_0036.stl
Save file: pedestrian_0014
File path: person_0037.stl
Save file: pedestrian_0015
File path: person_0039.stl
File path: person_0042.stl
Save file: pedestrian_0017
File path: person_0053.stl
File path: person_0055.stl
File path: person_0056.stl
File path: person_0057.stl
File path: person_0061.stl
File path: person_0064.stl
File path: person_0065.stl
Save file: pedestrian_0024
File path: person_0066.stl
File path: person_0067.stl
Save file: pedestrian_0026
File path: person_0068.stl
S